# Extended Neighbourhoods Control

This notebook is a control to which the first attempt at implementing Extended Neighbourhoods can be compared.

In [1]:
import numpy as np
import scipy.sparse as sp
from scipy.spatial.distance import cdist
import tensorflow as tf
import time

from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import sparse_categorical_accuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from spektral.data import Dataset, Graph
from spektral.data import BatchLoader
from spektral.datasets.mnist import MNIST
from spektral.layers import GCNConv, GlobalSumPool
from spektral.utils.sparse import sp_matrix_to_sp_tensor

In [2]:
# Parameters
batch_size = 64  # Batch size
epochs = 1000  # Number of training epochs
patience = 10  # Patience for early stopping
l2_reg = 5e-4  # Regularization rate for l2

# Import MNIST dataset in pixel format
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train, x_test = x_train.astype("float32")[...]/255.0, x_test.astype("float32")[...]/255.0

# Pixels with greater than 0.4x max brightness are considered 'on', all others are 'off'
x_train = np.where(0.4 < x_train, 1, 0)
x_test = np.where(0.4 < x_test, 1, 0)



# Graph Generation
# This class takes in the MNIST dataset in pixel form and converts it into a Graph format
# The details of this format can be found here: https://graphneural.network/data/#graph

In [3]:
class GenerateDataset(Dataset):
    def __init__(self, n_samples, data, labels, nbdsize =1, **kwargs):
        self.n_samples = n_samples
        self.data = data
        self.labels = labels
        self.nbdsize = nbdsize
        super().__init__(**kwargs)
    
    
    def read(self):
        def make_graph(ind):
            # Flatten the 28x28 grid into a 784 length array
            # This will be immediately undone in the next step but is being done here anyway as practice for future projects
            nodes = np.ndarray.flatten(self.data[ind])
            # Isolate the indices of the bright pixels
            bright = np.delete((np.where(nodes == 1)), -1)
            
            node_coords = []
            edges = []
            edges_2 = []
            
            # The divmod function returns the coordinates from the index in the 784-array
            # he quotient yields the y-coordinate and the remainder yields the x-coordinate
            for i in range(len(bright)):
                node_coords.append(divmod(bright[i],28))
            
            # This creates a matrix of the distances between the bright pixels
            DistMat = cdist(node_coords, node_coords)
            
            # This step iterates over pairs of bright pixels, and the indices of the pairs are added to 'edges' if they are within a certain distance
            # Selecting 1.5 will result edges being created between bright pixels that are within each others' neighbourhood-of-8
            # (Diagonal distance is sqrt(2) =~ 1.414)
            # The distance can be increased accordingly to include larger neighbourhoods
            for i in range(len(node_coords)):
                if self.nbdsize == 1:
                    for j in range(i+1, len(node_coords)):
                        if DistMat[i][j] <= 1.5:
                            edges.append((bright[i],bright[j]))
                elif self.nbdsize == 2:
                    for j in range(i+1, len(node_coords)):
                        if DistMat[i][j] <= 1.5:
                            edges.append((bright[i],bright[j]))
                        elif DistMat[i][j] <= 2.3:
                            edges_2.append((bright[i],bright[j]))
                                

            # Node features
            x = np.array(nodes, dtype=np.float32)

            # Edges
            r, c = zip(*edges)
            a = sp.csr_matrix((np.ones(len(r)), (np.array(r), np.array(c))), shape=(784, 784), dtype=np.float32)
            if self.nbdsize == 2:
                r_2, c_2 = zip(*edges_2)
                a_2 = sp.csr_matrix(((np.ones(len(r_2))/3), (np.array(r_2), np.array(c_2))), shape=(784, 784), dtype=np.float32)
                a = a + a_2
            
            # Labels
            y = self.labels[ind]
            
            # Counters and Diagnostics
            if ind == 0:
                print("Graph generation started")
            elif (ind%100) == 0:
                print(ind, "graphs generated")
                
            ind +=1   
            
            return Graph(x=x, a=a, y=y)
                    
        # We must return a list of Graph objects
        return [make_graph(index) for index in range(self.n_samples)]

In [4]:
# Generating and shuffling the data  
# Calling this generates the train, test, and validation datasets with randomized indices      
traindata = GenerateDataset(len(x_train), x_train, y_train, 1)
idxs = np.random.permutation(len(traindata))
split_val = int(0.85 * len(traindata))
idx_tr, idx_val = np.split(idxs, [split_val])
data_tr = traindata[idx_tr]
data_val = traindata[idx_val]

testdata = GenerateDataset(len(x_test), x_test, y_test, 1)
idx_test = np.random.permutation(len(testdata))
data_test = testdata[idx_test]



# Data loaders
# See https://graphneural.network/loaders/ for further information
loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = BatchLoader(data_val, batch_size=batch_size)
loader_te = BatchLoader(data_test, batch_size=batch_size)


# Build model
class Net(Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.conv1 = GCNConv(16, activation="elu", kernel_regularizer=l2(l2_reg))
        self.conv2 = GCNConv(32, activation="elu", kernel_regularizer=l2(l2_reg))
        self.flatten = GlobalSumPool()
        self.fc1 = Dense(512, activation="relu")
        self.fc2 = Dense(10, activation="softmax")  # MNIST has 10 classes

    def call(self, inputs):
        x, a = inputs
        x = self.conv1([x, a])
        x = self.conv2([x, a])
        output = self.flatten(x)
        output = self.fc1(output)
        output = self.fc2(output)

        return output

Graph generation started


C:\anaconda3\lib\site-packages\spektral\data\graph.py:67: UserWarning: x was automatically reshaped to (784, 1)
  warnings.warn(f"x was automatically reshaped to {x.shape}")


100 graphs generated
200 graphs generated
300 graphs generated
400 graphs generated
500 graphs generated
600 graphs generated
700 graphs generated
800 graphs generated
900 graphs generated
1000 graphs generated
1100 graphs generated
1200 graphs generated
1300 graphs generated
1400 graphs generated
1500 graphs generated
1600 graphs generated
1700 graphs generated
1800 graphs generated
1900 graphs generated
2000 graphs generated
2100 graphs generated
2200 graphs generated
2300 graphs generated
2400 graphs generated
2500 graphs generated
2600 graphs generated
2700 graphs generated
2800 graphs generated
2900 graphs generated
3000 graphs generated
3100 graphs generated
3200 graphs generated
3300 graphs generated
3400 graphs generated
3500 graphs generated
3600 graphs generated
3700 graphs generated
3800 graphs generated
3900 graphs generated
4000 graphs generated
4100 graphs generated
4200 graphs generated
4300 graphs generated
4400 graphs generated
4500 graphs generated
4600 graphs generat

36200 graphs generated
36300 graphs generated
36400 graphs generated
36500 graphs generated
36600 graphs generated
36700 graphs generated
36800 graphs generated
36900 graphs generated
37000 graphs generated
37100 graphs generated
37200 graphs generated
37300 graphs generated
37400 graphs generated
37500 graphs generated
37600 graphs generated
37700 graphs generated
37800 graphs generated
37900 graphs generated
38000 graphs generated
38100 graphs generated
38200 graphs generated
38300 graphs generated
38400 graphs generated
38500 graphs generated
38600 graphs generated
38700 graphs generated
38800 graphs generated
38900 graphs generated
39000 graphs generated
39100 graphs generated
39200 graphs generated
39300 graphs generated
39400 graphs generated
39500 graphs generated
39600 graphs generated
39700 graphs generated
39800 graphs generated
39900 graphs generated
40000 graphs generated
40100 graphs generated
40200 graphs generated
40300 graphs generated
40400 graphs generated
40500 graph

In [5]:
# Create model
model = Net()
optimizer = Adam()
loss_fn = SparseCategoricalCrossentropy()

# Training function
@tf.function
def train_on_batch(inputs, target):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(target, predictions) + sum(model.losses)
        acc = tf.reduce_mean(sparse_categorical_accuracy(target, predictions))

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss, acc


# Evaluation function
def evaluate(loader):
    step = 0
    results = []
    for batch in loader:
        step += 1
        inputs, target = batch
        predictions = model(inputs, training=False)
        loss = loss_fn(target, predictions)
        acc = tf.reduce_mean(sparse_categorical_accuracy(target, predictions))
        results.append((loss, acc, len(target)))  # Keep track of batch size
        if step == loader.steps_per_epoch:
            results = np.array(results)
            return np.average(results[:, :-1], 0, weights=results[:, -1])


# Setup training
best_val_loss = 99999
current_patience = patience
step = 0

In [6]:
# Training loop
results_tr = []
start_time = time.process_time()
e_num = 1
for batch in loader_tr:
    step += 1

    # Training step
    inputs, target = batch
    loss, acc = train_on_batch(inputs, target)
    results_tr.append((loss, acc, len(target)))

    if step == loader_tr.steps_per_epoch:
        results_va = evaluate(loader_va)
        if results_va[0] < best_val_loss:
            best_val_loss = results_va[0]
            current_patience = patience
            results_te = evaluate(loader_te)
        else:
            current_patience -= 1
            if current_patience == 0:
                print("Early stopping")
                break

        # Print results
        results_tr = np.array(results_tr)
        results_tr = np.average(results_tr[:, :-1], 0, weights=results_tr[:, -1])
        end_time = time.process_time()
        process_time = end_time - start_time
        print(
            "Train loss: {:.4f}, acc: {:.4f} | "
            "Valid loss: {:.4f}, acc: {:.4f} | "
            "Test loss: {:.4f}, acc: {:.4f}".format(
                *results_tr, *results_va, *results_te
            )
        )
        print(
            "Epoch Number: {:.0f}, Epoch Time: {:.0f} minutes {:.0f} seconds ".format(
                e_num, (process_time // 60), (process_time % 60)
            )
        )

        # Reset epoch
        results_tr = []
        step = 0
        e_num += 1
        start_time = time.process_time()

C:\anaconda3\lib\site-packages\spektral\data\utils.py:206: UserWarning: `x` isn't a recognized object; `shuffle` is not guaranteed to behave correctly. E.g., non-numpy array/tensor objects with view semantics may contain duplicates after shuffling.
  shuffle_inplace(*data)


Train loss: 3.6429, acc: 0.2491 | Valid loss: 1.7825, acc: 0.3376 | Test loss: 1.7823, acc: 0.3395
Epoch Number: 1 Epoch Time: 3 minutes 15 seconds 
Train loss: 1.9138, acc: 0.3303 | Valid loss: 1.7287, acc: 0.3638 | Test loss: 1.7094, acc: 0.3701
Epoch Number: 2 Epoch Time: 2 minutes 44 seconds 
Train loss: 1.7494, acc: 0.3599 | Valid loss: 1.7104, acc: 0.3767 | Test loss: 1.7048, acc: 0.3707
Epoch Number: 3 Epoch Time: 2 minutes 44 seconds 
Train loss: 1.6922, acc: 0.3746 | Valid loss: 1.6434, acc: 0.3728 | Test loss: 1.6252, acc: 0.3755
Epoch Number: 4 Epoch Time: 2 minutes 46 seconds 
Train loss: 1.6682, acc: 0.3807 | Valid loss: 1.5991, acc: 0.3981 | Test loss: 1.5892, acc: 0.4025
Epoch Number: 5 Epoch Time: 2 minutes 47 seconds 
Train loss: 1.6575, acc: 0.3828 | Valid loss: 1.6574, acc: 0.3811 | Test loss: 1.5892, acc: 0.4025
Epoch Number: 6 Epoch Time: 2 minutes 23 seconds 
Train loss: 1.6469, acc: 0.3846 | Valid loss: 1.6528, acc: 0.3876 | Test loss: 1.5892, acc: 0.4025
Epoch N

Train loss: 1.3906, acc: 0.4818 | Valid loss: 1.3775, acc: 0.4882 | Test loss: 1.3510, acc: 0.4891
Epoch Number: 56 Epoch Time: 2 minutes 56 seconds 
Train loss: 1.3901, acc: 0.4808 | Valid loss: 1.3814, acc: 0.4787 | Test loss: 1.3510, acc: 0.4891
Epoch Number: 57 Epoch Time: 2 minutes 48 seconds 
Train loss: 1.3951, acc: 0.4773 | Valid loss: 1.3643, acc: 0.4892 | Test loss: 1.3642, acc: 0.4934
Epoch Number: 58 Epoch Time: 3 minutes 15 seconds 
Train loss: 1.3923, acc: 0.4806 | Valid loss: 1.3559, acc: 0.4870 | Test loss: 1.3534, acc: 0.4805
Epoch Number: 59 Epoch Time: 3 minutes 15 seconds 
Train loss: 1.3904, acc: 0.4833 | Valid loss: 1.3508, acc: 0.4936 | Test loss: 1.3470, acc: 0.4972
Epoch Number: 60 Epoch Time: 3 minutes 18 seconds 
Train loss: 1.3828, acc: 0.4843 | Valid loss: 1.3470, acc: 0.4983 | Test loss: 1.3440, acc: 0.4971
Epoch Number: 61 Epoch Time: 3 minutes 13 seconds 
Train loss: 1.3772, acc: 0.4861 | Valid loss: 1.3431, acc: 0.4902 | Test loss: 1.3287, acc: 0.4890
E

Train loss: 1.2736, acc: 0.5284 | Valid loss: 1.2962, acc: 0.5181 | Test loss: 1.2195, acc: 0.5521
Epoch Number: 111 Epoch Time: 2 minutes 9 seconds 
Train loss: 1.2698, acc: 0.5320 | Valid loss: 1.2822, acc: 0.5098 | Test loss: 1.2195, acc: 0.5521
Epoch Number: 112 Epoch Time: 2 minutes 8 seconds 
Train loss: 1.2721, acc: 0.5283 | Valid loss: 1.2584, acc: 0.5281 | Test loss: 1.2195, acc: 0.5521
Epoch Number: 113 Epoch Time: 2 minutes 8 seconds 
Train loss: 1.2757, acc: 0.5274 | Valid loss: 1.2393, acc: 0.5407 | Test loss: 1.2195, acc: 0.5521
Epoch Number: 114 Epoch Time: 2 minutes 9 seconds 
Train loss: 1.2775, acc: 0.5292 | Valid loss: 1.3119, acc: 0.5064 | Test loss: 1.2195, acc: 0.5521
Epoch Number: 115 Epoch Time: 2 minutes 9 seconds 
Train loss: 1.2627, acc: 0.5362 | Valid loss: 1.2429, acc: 0.5249 | Test loss: 1.2195, acc: 0.5521
Epoch Number: 116 Epoch Time: 2 minutes 9 seconds 
Train loss: 1.2723, acc: 0.5301 | Valid loss: 1.2571, acc: 0.5216 | Test loss: 1.2195, acc: 0.5521
E